# 라이브러리를 사용해서 클러스터링을 해보자

read_data_from_folder

In [1]:
import os
import numpy as np
import pandas as pd 
from krx_fr.preprocess import tools, scaler
from krx_fr.cluster.kmeans import MyKmeans

In [2]:
kind_data = tools.read_data_from_folder("C:\\Jehyeon\\2022_Winter_KRX\\KRX\\Use_data\\KIND\\")
financial_data = tools.read_data_from_folder("C:\\Jehyeon\\2022_Winter_KRX\\KRX\\Use_data\\재무\\")

kind_data2 = tools.dict_data_preprocess(kind_data, function=np.sum, window_size='Y',fillna=True)
financial_data2 = tools.dict_data_preprocess(financial_data, function=np.mean, window_size='Y',fillna=False)

In [45]:
d = pd.read_csv("기업코드_시장.csv", index_col=0)#.drop("시장", axis=1)
d.sort_values("기업공시코드").iloc[:50]

,기업공시코드,시장
1850,A000020,코스피
3010,A000020,코스닥
2208,A000030,코스피
389,A000040,코스피
2726,A000040,코스닥
1704,A000050,코스피
3174,A000050,코스닥
3104,A000060,코스닥
1807,A000060,코스피
1212,A000070,코스피


In [38]:
kind_data["BW금액_전체시장"].T.reset_index()

공시일,index,2012-01-03 00:00:00,2012-01-04 00:00:00,2012-01-09 00:00:00,2012-01-10 00:00:00,2012-01-11 00:00:00,2012-01-17 00:00:00,2012-01-20 00:00:00,2012-02-01 00:00:00,2012-02-02 00:00:00,...,2022-08-05 00:00:00,2022-09-05 00:00:00,2022-09-13 00:00:00,2022-10-12 00:00:00,2022-10-26 00:00:00,2022-11-23 00:00:00,2022-12-01 00:00:00,2022-12-13 00:00:00,2022-12-19 00:00:00,2022-12-27 00:00:00
0,A000040,0.0,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,A000220,0.0,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,A000250,0.0,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,A000300,0.0,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,A000390,0.0,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
520,A900050,0.0,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
521,A900080,0.0,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
522,A900100,0.0,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
523,A900110,0.0,0.0,0.0,0.0,0.0,0.0,2.000000e+10,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [42]:
d1 = d.loc[d["시장"]=="코스피"]
d1.sort_values("기업공시코드")
d2 = kind_data["BW금액_전체시장"].T.reset_index()

pd.merge(d2,d1, left_on="index", right_on="기업공시코드").set_index("기업공시코드")

,index,2012-01-03 00:00:00,2012-01-04 00:00:00,2012-01-09 00:00:00,2012-01-10 00:00:00,2012-01-11 00:00:00,2012-01-17 00:00:00,2012-01-20 00:00:00,2012-02-01 00:00:00,2012-02-02 00:00:00,...,2022-09-05 00:00:00,2022-09-13 00:00:00,2022-10-12 00:00:00,2022-10-26 00:00:00,2022-11-23 00:00:00,2022-12-01 00:00:00,2022-12-13 00:00:00,2022-12-19 00:00:00,2022-12-27 00:00:00,시장
기업공시코드,,,,,,,,,,,,,,,,,,,,,
A000040,A000040,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,코스피
A000220,A000220,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,코스피
A000300,A000300,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,코스피
A000390,A000390,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,코스피
A000420,A000420,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,코스피
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
A161890,A161890,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,코스피
A180640,A180640,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,코스피
A214330,A214330,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,코스피


In [36]:
d1 = d.loc[d["시장"]=="코스닥"]
d1.sort_values("기업공시코드")
d2 = kind_data["BW금액_전체시장"].T

pd.merge(d2,d1, left_index=True, right_on="기업공시코드").set_index("기업공시코드").shape

(515, 560)

In [3]:
tools.dict_data_market_screener(kind_data)["BW금액_전체시장"]

,2012-01-03 00:00:00,2012-01-04 00:00:00,2012-01-09 00:00:00,2012-01-10 00:00:00,2012-01-11 00:00:00,2012-01-17 00:00:00,2012-01-20 00:00:00,2012-02-01 00:00:00,2012-02-02 00:00:00,2012-02-07 00:00:00,...,2022-09-13 00:00:00,2022-10-12 00:00:00,2022-10-26 00:00:00,2022-11-23 00:00:00,2022-12-01 00:00:00,2022-12-13 00:00:00,2022-12-19 00:00:00,2022-12-27 00:00:00,기업공시코드,시장
2726,0.0,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,A000040,코스닥
2569,0.0,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,A000220,코스닥
232,0.0,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,A000250,코스닥
2797,0.0,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,A000300,코스닥
3007,0.0,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,A000390,코스닥
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2621,0.0,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,A900050,코스닥
109,0.0,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,A900080,코스닥
179,0.0,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,A900100,코스닥
10,0.0,0.0,0.0,0.0,0.0,0.0,2.000000e+10,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,A900110,코스닥


In [ ]:
kind_data

In [ ]:
dl = tools.dict_data_calculate_vol(financial_data, name="최대주주비율", return_original=True)

In [ ]:
financial_data2 = tools.dict_data_preprocess(dl, function=np.mean, window_size='Y',fillna=False)
financial_data2["2014-01-01"].iloc[300:350]

In [ ]:
concat_data = tools.dict_data_concat(kind_data2,financial_data2, dropna=False)

print(len(concat_data["2012-01-01"].columns))

In [ ]:
tools.dict_data_calculate_vol(concat_data, "최대주주비율", return_original=True)["2012-01-01"]

In [ ]:
plus_data = tools.dict_data_plus(concat_data, plus_one="BW금액_전체시장",plus_two="CB금액_전체시장", return_origianl=False)

plus_data["2012-01-01"].columns

In [ ]:
plus_data2 = tools.dict_data_plus(plus_data, plus_one="상호변경카운트_전체시장",plus_two="업종변경카운트_전체시장", return_origianl=False)
plus_data2["2012-01-01"].columns

In [ ]:
plus_data3 = tools.dict_data_plus(plus_data2, plus_one="불성실공시법인지정카운트_전체시장",plus_two="조회공시요구카운트_전체시장", return_origianl=False)
plus_data3["2012-01-01"].columns

In [ ]:
plus_data4 = tools.dict_data_plus(plus_data3, plus_one="불성실공시법인지정카운트_전체시장+조회공시요구카운트_전체시장",plus_two="최대주주변경카운트_전체시장", return_origianl=False)
plus_data4["2012-01-01"].columns

In [ ]:
plus_data5 = tools.dict_data_plus(plus_data4, plus_one="자기주식처분결정금액_전체시장",plus_two="특수관계인에대한자금대여금액_전체시장", return_origianl=False)
plus_data5["2012-01-01"].columns

In [ ]:
divide_data = tools.dict_data_divide(plus_data5,divide_what=['자기주식처분결정금액_전체시장+특수관계인에대한자금대여금액_전체시장',
       'BW금액_전체시장+CB금액_전체시장', '당기순이익', '매출채권', '매출총이익',
       '영업이익', '영업현금흐름', '판매 및 관리비'], divide_into="총자본", return_origianl=False)
print(len(divide_data["2012-01-01"].columns))
divide_data["2012-01-01"].columns

In [ ]:
drop_data = tools.dict_data_drop(divide_data, dropna=True)
scaled_data = tools.dict_data_scale(drop_data, scaler=scaler.sklearn_maxabs)

In [ ]:
scaled_data["2022-01-01"].shape

In [ ]:
# 클러스터링 할 sample data 하나 생성
sample_2022 = scaled_data["2022-01-01"]
company = scaled_data["2022-01-01"].index.to_list()

In [ ]:
sample_2022.columns

In [ ]:
import pandas as pd
df = pd.DataFrame({"a":[1,2,3,4,5], "b":[10,20,30,40,50], "c":[0,0,0,0,0]})
scaler.sklearn_minmax(df)

In [ ]:
import matplotlib.pyplot as plt 

plt.figure(figsize=(20,12))
plt.scatter(range(len(sample_2022["최대주주비율"].values)),sample_2022["최대주주비율"].values);

In [ ]:
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=4, n_init=100, max_iter=10000)
kmeans.fit(sample_2022)

cluster_df = pd.DataFrame(data=np.c_[company,kmeans.labels_], columns=["기업공시코드","클러스터"])
cluster_df["클러스터"].value_counts()

거리요소로 분해하기 위해선 kmeans를 써주는 게 중요하다

## 클러스터링 라이브러리

In [ ]:
cluster_model = MyKmeans(sample_2022)
cluster_model.set_params()
cluster_model.find_optimal_k(max_k=10, max_sample=20, optimize_method="silhouette")
#cluster_model.run_kmean(5,)
#cluster_model.

In [ ]:
cluster_model.find_optimal_initp(5,20, "silhouette")

In [ ]:
k_mean_dict = cluster_model.run_kmean()

In [ ]:
cluster_model.distance_decomposition(k_mean_dict["model"])[0][1]

In [ ]:
for i, score_list in dict_sil.items():
    if i == 0:
        start_ar = np.array(score_list)
    elif i ==1:
        concat_ar = np.c_[start_ar, score_list]
    else:
        concat_ar = np.c_[concat_ar,score_list]

mean_ar = concat_ar.mean(axis=1)
print(mean_ar)
print("최고 점수의 인덱스는: ", mean_ar.argmax())

In [ ]:
for i, score in dict_sil.items():
    ar = np.array(score)
    print("{}번째 최대값의 인덱스는 ")

In [ ]:
test = MyKmeans.distance_decomposition(kmeans)

In [ ]:
test[0][1]

In [ ]:
MyKmeans.visualize(sample_2022,kmeans)

In [ ]:
# 클러스터링을 해보자
from sklearn.cluster import AgglomerativeClustering

agg = AgglomerativeClustering(n_clusters=5)
agg.fit(sample_2022)

clus = pd.DataFrame(np.c_[company,agg.labels_], columns= ["기업공시코드","클러스터"])
clus["클러스터"].value_counts()

In [ ]:
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=5, n_init=100, max_iter=10000)
kmeans.fit(sample_2022)

cluster_df = pd.DataFrame(data=np.c_[company,kmeans.labels_], columns=["기업공시코드","클러스터"])
cluster_df["클러스터"].value_counts()

In [ ]:
from sklearn.cluster import DBSCAN

db = DBSCAN(min_samples=40, n_jobs=-1)
db.fit(sample_2022)

label_db = db.labels_
cluster_db = pd.DataFrame(data=np.c_[company, label_db], columns=["기업공시코드","클러스터"])
cluster_db["클러스터"].value_counts()